# TP2 - EA3
Diseñar, simular e implemetar un amplificador de microondas de baja señal para máxima transferencia de energía (adaptación de impedancias de entrada y salida)
Los requisitos son:
- $Z_{in}$ = 50 $\Omega$
- $Z_L$ = 50 $\Omega$
- FR4 $\rightarrow \epsilon_r = 4.5$
- El transistor elegido es BFP 450
 A continuacion se cargan en un arreglo los parámetros S en base a la tensión y corriente de polarización seleccionadas para el transistor

### Se procede a cargar los datos del archivo s2p para poder trabajar con ellos

In [1]:
import os
import numpy as np
import skrf as skrf
from sympy import symbols, Eq, solve

#Calcula los parámetros S en una frecuencia dada para cada archivo .s2p en las carpetas especificadas.
def calcular_parametros_s(frecuencia, carpetas):
    s_parameters = {}
    
    # Convertir frecuencia de GHz a Hz
    frecuencia *= 1e9                                  
    
    for carpeta in carpetas:
        archivos_s2p = [f for f in os.listdir(carpeta) if f.endswith('.s2p')]
        for archivo_s2p in archivos_s2p:
            ruta_archivo = os.path.join(carpeta, archivo_s2p)
            red = skrf.network.Network(ruta_archivo)
            
            # Encontrar la frecuencia más cercana en el archivo .s2p
            indice_frecuencia = np.argmin(np.abs(red.f - frecuencia))
                        
            # Acceder a los parámetros S en la frecuencia encontrada
            if archivo_s2p not in s_parameters:
                s_parameters[archivo_s2p] = red.s[indice_frecuencia]
    
    return s_parameters

### Se calcula el determinante para cada conjunto de parametros S

In [2]:
def calcular_delta(parametros_s):
    deltas = {}
    for archivo, parametros in parametros_s.items():
        delta = (parametros[0, 0] * parametros[1, 1]) - (parametros[0, 1] * parametros[1, 0])
        deltas[archivo] = delta

    return deltas

### Se calcula el parametro k de cada conjunto y se filtran los k > 1

In [3]:
def calcular_k(s_parametros, deltas):
    ks = {}
    polarizacion_k_bueno = {}
    for archivo, parametros in s_parametros.items():
        S11 = parametros[0, 0]
        S12 = parametros[0, 1]
        S21 = parametros[1, 0]
        S22 = parametros[1, 1]
        abs_delta = abs(deltas[archivo])
        
        # Calcular el parámetro k
        numerador = (1 - abs(S11)**2 - abs(S22)**2 + abs_delta**2)
        denominador = 2 * abs(S12 * S21)
        k = numerador / denominador
        
        ks[archivo] = k
        if k > 1:
            # Guardar la polarización si k es mayor que 1
            polarizacion_k_bueno[archivo] = {'k': k}
    
    return ks, polarizacion_k_bueno


### Se calculan los coeficientes B1, B2, C1 y C2 y las impedancias Zin y Zout

In [4]:
def calcular_impedancias(Zo, s_parameters, polarizaciones_buenas_k, ic_filter, vce_filter, frecuencia):
    frecuencia *= 1e9  
    impedancias = []

    for nombre_archivo, polarizacion in polarizaciones_buenas_k.items():
        # Obtener el valor de IC del nombre del archivo
        ic  = float(nombre_archivo.split('_IC_') [1].split('mA')[0])
        vce = float(nombre_archivo.split('_VCE_')[1].split('V')[0])

        # Filtro de polarizaciones
        if ic == ic_filter and vce == vce_filter:
        # if ic < ic_filter:
            # Obtener los parámetros S de la polarización actual
            parametros_s = s_parameters[nombre_archivo]

            # Asignar los valores de los parámetros S a las variables correspondientes
            S11 = parametros_s[0, 0]
            S12 = parametros_s[0, 1]
            S21 = parametros_s[1, 0]
            S22 = parametros_s[1, 1]
            det = S11 * S22 - S12 * S21                 # Determinate

            # Cálculo de B1, B2, C1, C2
            B1 = 1 + abs(S11)**2 - abs(S22)**2 - abs(det)**2
            B2 = 1 + abs(S22)**2 - abs(S11)**2 - abs(det)**2
            C1 = S11 - det * np.conj(S22)
            C2 = S22 - det * np.conj(S11)
            
            # Cálculo de gamma in
            gamma_in = (B1 - np.sqrt((B1**2) - (4 * (abs(C1)**2)))) / (2 * abs(C1))
            gamma_in_rect = gamma_in * np.cos(np.angle(C1)) + gamma_in * np.sin(np.angle(C1)) * 1j

            # Cálculo de gamma out
            gamma_out = (B2 - np.sqrt((B2**2) - (4 * (abs(C2)**2)))) / (2 * abs(C2))
            gamma_out_rect = gamma_out * np.cos(np.angle(C2)) + gamma_out * np.sin(np.angle(C2)) * 1j

            "Adaptacion Entrada"
            # Cálculo impedancia de entrada serie
            Z_in = Zo * ((1 + gamma_in_rect) / (1 - gamma_in_rect))
            print("Admitancia: ", 1/Z_in)
            # Cálculo impedancia de entrada paralelo
            Z_in_p = np.real(Z_in) * (1+(np.imag(Z_in)/np.real(Z_in))**2) + 1j*np.imag(Z_in) * (1+(np.real(Z_in)/np.imag(Z_in))**2)

            # Cálculo de C_in adaptacion
            C_in =1/(2*np.pi*frecuencia*np.imag(Z_in_p))

            # Cálculo de transformador lamda entrada
            Z_in_trafo = np.sqrt(Zo*np.real(Z_in_p))

            
            "Adaptacion Salida"
            # Cálculo impedancia de salida serie
            Z_out = Zo * ((1 + gamma_out_rect) / (1 - gamma_out_rect)) 
            

            # Cálculo impedancia de salida paralelo
            Z_out_p = np.real(Z_out) * (1+(np.imag(Z_out)/np.real(Z_out))**2) + 1j*np.imag(Z_out) * (1+(np.real(Z_out)/np.imag(Z_out))**2)

            # Cálculo de transformador lamda salida
            Z_out_trafo = np.sqrt(Zo*np.real(Z_out))

            # Cálculo de L_out paralelo
            L_out_p = 1j*Z_out_trafo**2/np.imag(Z_out)

            # Cálculo de C_out adaptacion
            C_out =1/(2*np.pi*frecuencia*np.imag(-L_out_p))

            # Agregar las impedancias a la lista impedancias
            impedancias.append((Z_in_trafo, Z_out_trafo, C_in, C_out))
            # impedancias.append( Z_out_trafo)

    
               
            # Imprimir los resultados para cada configuración
            print(f"\033[1m\033[31mPolarización: Vce = {vce} [V] - Ic = {ic} [mA]\033[0m")
            print(f"S11: {S11:.4f}, S12: {S12:.4f}, S21: {S21:.4f}, S22: {S22:.4f}")
            print(f"B1 =", "{:.4f}".format(abs(B1)), "∡", "{:.4f}".format(np.degrees(np.angle(B1))), "°")
            print(f"B2 =", "{:.4f}".format(abs(B2)), "∡", "{:.4f}".format(np.degrees(np.angle(B2))), "°")
            print(f"C1 =", "{:.4f}".format(abs(C1)), "∡", "{:.4f}".format(np.degrees(np.angle(C1))), "°")
            print(f"C2 =", "{:.4f}".format(abs(C2)), "∡", "{:.4f}".format(np.degrees(np.angle(C2))), "°")
            print("Γᵢₙ      = ","{:.4f}".format(gamma_in), "∡", "{:.4f}".format(np.degrees(np.angle(C1))),"°")
            print("Γᵢₙ_rect = ","{:.4f}".format(gamma_in_rect))
            print("Γₒᵤ      = ","{:.4f}".format(gamma_out), "∡", "{:.4f}".format(np.degrees(np.angle(C2))),"°")
            print("Γₒᵤ_rect = ","{:.4f}".format(gamma_out_rect))
            print()
            print("ADAPTACION ENTRADA")
            print("Z_in     = ","{:.4f}".format(Z_in))
            
            print("C_in     = ","{:.4e}".format(C_in), "F") 
            print("Z_in_trafo= ","{:.4f}".format(Z_in_trafo))
            print()
            print("ADAPTACION SALIDA")
            print("Z_out    = ","{:.4f}".format(Z_out))  
            print("C_out    = ","{:.4e}".format(C_out), "F")
            print("Z_out_trafo= ","{:.4f}".format(Z_out_trafo))
            print()

            
    return impedancias




### Diseño de microtiras

In [5]:
def microtiras(Z, er, H, t, frecuencia, tipo, C = None):

    A = ((Z / 60) * np.sqrt((er + 1) / 2)) + ((er - 1) / (er + 1)) * (0.23 + (0.11 / er))
    B = 377 * np.pi / (2 * Z * np.sqrt(er))
    print("A: ", A, "B: ", B)

    WH1 = (8 * np.exp(A)) / (np.exp(2 * A) - 2)                                                                     # Para W/H < 2
    WH2 = (2/np.pi) * ((B - 1 - np.log(2*B - 1)) + (((er - 1) / (2*er)) * (np.log(B - 1) + 0.39 - (0.61 / er))))    # Para W/H > 2
    #print("W/H1: ", WH1, "W/H2", WH2)

    if(WH1 < 2):
        WH = WH1
    
    else:
        WH = WH2
        
    print("WH: ", WH)
    
    # Ancho de la microtira
    W = WH * H 
    print("W: ", W)

    # Ancho efectivo de la microtira
    if(WH < (1/2) * np.pi):
        We = W + (t/np.pi) * (1 + (np.log((4*np.pi*W)/t)))
    
    else:
        We = W + (t/np.pi) * (1 + (np.log((2*H)/t)))

    # Permitividad relativa efectiva e impedancia efectiva
    if (WH < 1):
        er_e = (er + 1)/2 + ((er - 1)/2) * (1/(np.sqrt(1 + (12/WH))) + 0.04 * (1 - WH)**2)
        Zo_e = (60/np.sqrt(er_e)) * np.log((8*H)/W + W/(4*H))
    
    
    else:
        er_e = (er + 1)/2 + ((er - 1)/2) * (1/(np.sqrt(1 + (12/WH))))
        Zo_e = ((120*np.pi)/np.sqrt(er_e))/(WH + 1.393 + 0.667*np.log(WH + 1.444))
        
    print("Zo_e = ", Zo_e)
    print("er_e = ", er_e)
    
    # Lambda 
    #print(frecuencia*1000)
    lambda_  = 300/(frecuencia*1000)
    
   # print("lambda = ", lambda_)
    lambda_p = lambda_/np.sqrt(er_e)
    print("lambda_p = ", lambda_p)
    print("Lambda_e/4: ", lambda_p/4*1000, "mm")
    
    
    if(tipo == "T"):
        # Largo de la microtira
        largo = lambda_p/4

    elif(tipo == "C"):
        X_c = 1/(2*np.pi*frecuencia*1e9*C)
        Z_d   = X_c
        betta = 2*np.pi/lambda_p
        d     = (np.arctan(Z/Z_d)) / betta
        angulo = betta * d * 180/(np.pi)
        
        print("Ángulo = ", angulo)
        
        area_trapecio = ((1 + We)/2)* 3
        area_cap = d * We * 1000                      # Multiplica por 1000 por pasaje de unidad
        area_cap_in = area_cap - area_trapecio
        largo = (area_cap_in/We)/1000                 # Divide por 1000 por pasaje de unidad

    else:
        X_L = 2*np.pi*frecuencia*1e9*C                # C = inductor
        Z_d   = X_L
        betta = 2*np.pi/lambda_p
        d     = (np.arctan(Z/Z_d)) / betta
        angulo = betta * d * 180/(np.pi)

        
        # print("X_l = ", X_L)
        # area_trapecio = ((1 + We)/2)* 3
        # area_cap = d * We * 1000                      # Multiplica por 1000 por pasaje de unidad
        # area_cap_in = area_cap - area_trapecio
        # largo = (area_cap_in/We)/1000                 # Divide por 1000 por pasaje de unidad
        largo = d
    
    return We, largo
        


### Polarización

In [6]:
def polarizacion(ic, vce, Vcc):
    # Definir incógnitas
    Vbb, Rc, Rb = symbols('Vbb Rc Rb')

    # Definir constante
    Ic = ic / 1000  # Convertir mA a A

    # Definir ecuaciones
    eq1 = Eq(Vcc, Ic * Rc - (Ic / 100) * Rb - Vbb)
    eq2 = Eq(Vcc, Ic * Rc - vce)
    eq3 = Eq(Vbb, (Ic / 100) * Rb - 0.7)

    # Resolver el sistema de ecuaciones
    solucion = solve((eq1, eq2, eq3), (Vbb, Rc, Rb))
    print(solucion)



### Ejemplo de uso

In [7]:
def main():
    frecuencia = 2.2                # GHz
    Zo         = 50                 # Impedancia  
    carpetas   = ["SPAR/BFP640"]    # Rutas a las carpetas que contienen los archivos .s2p
    ic_filter  = 35               # Corriente para filtrar polaricaciones
    vce_filter = 1.5
    er         = 4.3                # Permitividad relativa
    H          = 1.6                # Espesor del dielectrico - sustrato [mm]
    t          = 0.05               # Espesor del cobre [mm]
    Vcc        = 5 

    polarizacion(ic_filter, vce_filter, Vcc)
    s_parameters             = calcular_parametros_s(frecuencia, carpetas)
    deltas                   = calcular_delta(s_parameters)
    ks, polarizacion_k_bueno = calcular_k(s_parameters, deltas)
    impedancias              = calcular_impedancias(Zo, s_parameters, polarizacion_k_bueno, ic_filter, vce_filter, frecuencia)
    # print(impedancias)
    
    print("\033[1;32mMicrotira: transformador λ/4 entrada\033[0m")
    W_trafo_in, L_trafo_in = microtiras(impedancias[0][0], er, H, t, frecuencia, "T")       # impedancias[polarización][Z_deseado]
    print("W_trafo_in = ", W_trafo_in, "mm - Largo = ", L_trafo_in*1000, " mm")

    print()

    print("\033[1;32mMicrotira: transformador λ/4 salida\033[0m")
    W_trafo_out, L_trafo_out = microtiras(impedancias[0][1], er, H, t, frecuencia, "T")
    print("W_trafo_in = ", W_trafo_out, "mm - Largo = ", L_trafo_out*1000, " mm")
    
    print("\033[1;32mMicrotira: capacitor de entrada\033[0m")
    W_C_in, L_C_in = microtiras(Zo, er, H, t, frecuencia,  "C", impedancias[0][2])
    print("W_cap_in = ", W_C_in, "mm - Largo = ", L_C_in*1000, " mm")

    print()
    print("\033[1;32mMicrotira: capacitor de salida\033[0m")
    W_C_out, L_C_out = microtiras(Zo, er, H, t, frecuencia, "C", impedancias[0][3])
    print("W_cap_out = ", W_C_out, "mm - Largo = ", L_C_out*1000, " mm")


    print()
    print("\033[1mCorrección microtiras después de prueba 1\033[0m")
    print("\033[1;32mMicrotira: capacitor de entrada\033[0m")
    W_C_in, L_C_in = microtiras(Zo, er, H, t, frecuencia,  "C", 2.657e-12)
    print("W_cap_in = ", W_C_in, "mm - Largo = ", L_C_in*1000, " mm")

    print()
    print("\033[1;32mMicrotira: capacitor de salida\033[0m")
    W_C_out, L_C_out = microtiras(Zo, er, H, t, frecuencia, "C", 2.508e-12)
    print("W_cap_out = ", W_C_out, "mm - Largo = ", L_C_out*1000, " mm")
    
    
    print()
    print("\033[1mParámetros de choque \033[0m")
    print("\033[1;32mMicrotira: capacitor de desacople\033[0m")
    W_C_decoup, L_C_decoup = microtiras(40, er, H, t, frecuencia,  "C", 10e-12)
    print("W_cap_in = ", W_C_decoup, "mm - Largo = ", L_C_decoup*1000, " mm")

    print()
    print("\033[1;32mMicrotira: inductor de choque\033[0m")
    W_L_ch, L_L_ch = microtiras(60, er, H, t, frecuencia, "L", 100e-9)
    print("W_cap_out = ", W_L_ch, "mm - Largo = ", L_L_ch*1000, " mm")
    

main()

{Vbb: 0.400000000000000, Rc: 185.714285714286, Rb: 3142.85714285714}
Admitancia:  (0.09302735157117421-0.061356444674881404j)
Polarización: Vce = 1.5 [V] - Ic = 35.0 [mA]
S11: -0.3417+0.0935j, S12: 0.0322+0.0452j, S21: 3.0167+8.7612j, S22: 0.0842-0.1385j
B1 = 0.8872 ∡ 0.0000 °
B2 = 0.6887 ∡ 0.0000 °
C1 = 0.4245 ∡ 168.4587 °
C2 = 0.3193 ∡ -47.7208 °
Γᵢₙ      =  0.7416 ∡ 168.4587 °
Γᵢₙ_rect =  -0.7267+0.1484j
Γₒᵤ      =  0.6750 ∡ -47.7208 °
Γₒᵤ_rect =  0.4541-0.4994j

ADAPTACION ENTRADA
Z_in     =  7.4909+4.9406j
C_in     =  4.4387e-12 F
Z_in_trafo=  23.1835

ADAPTACION SALIDA
Z_out    =  49.7198-91.2328j
C_out    =  2.6549e-12 F
Z_out_trafo=  49.8597

Microtira: transformador λ/4 entrada
A:  0.7881366210232 B:  12.318202119496421
WH:  5.845248922879522
W:  9.352398276607236
Zo_e =  23.22133174742617
er_e =  3.594330777399059
lambda_p =  0.07192660357057921
Lambda_e/4:  17.981650892644804 mm
W_trafo_in =  9.434504450962221 mm - Largo =  17.981650892644804  mm

Microtira: transformador λ/